# Pattern matching
https://docs.microsoft.com/en-us/dotnet/csharp/fundamentals/functional/pattern-matching
https://docs.microsoft.com/en-us/dotnet/csharp/language-reference/operators/patterns#relational-patterns

## Declaration and type patterns
Declaration and type patterns are used to check if a variable is compatible with a certain type. With declaration pattern you can also declare a new local variable.

### Null checks

Declaration pattern only ever returns true if the expression is **not null**.  This example checks i for null AND converts it to underlying non-nullable `int` number. This is a declaration pattern.

In [ ]:
int? i = 5;
if (i is int number)
    Console.WriteLine($"I converted nullable int i to it's underlying non-nullable int number: {number}");
else
    Console.WriteLine("i is null");


I converted nullable int i to it's underlying non-nullable int number: 5


This next example takes a boxed `string` and converts it back into a `string` if it is compatible.

In [ ]:
object o = "Test";
if (o is string s) Console.WriteLine(s);

Test


### Type tests

Pattern matching can check if a variable matches a given type. This example tests if the variable implements the `IList<T>` interface. It also (which is always true for pattern matching) checks for null.

In [ ]:
public static T MidPoint<T>(IEnumerable<T> sequence)
{
    if (sequence is IList<T> list) // Check for null, check if matches IList<T> and assign to new variable ´list´
    {
        Console.WriteLine("Finding midpoint for ILists");
        return list[list.Count / 2];
    }
    else if (sequence is null)
    {
        throw new ArgumentNullException(nameof(sequence), "Sequence cannot be null");
    }
    else 
    {
        Console.WriteLine("Finding midpoint for IEnumerables");
        int halfLength = sequence.Count() / 2 - 1;
        if (halfLength < 0) halfLength = 0;
        return sequence.Skip(halfLength).First();
    }
}

var l = new List<string>() { "One", "Two", "Three" };
Console.WriteLine(MidPoint<string>(l));


Finding midpoint for ILists
Two


A type check isn't just true when the type matches but also when the type derives from T or implements T or another implicit reference conversion exists from it to T.
This is demonstrated below:

In [ ]:
var numbers = new int[] { 1, 2, 3, 4 };
Console.WriteLine(GetTypeLabel(numbers));
var chars = new List<char> { 'a', 'b', 'c' };
Console.WriteLine(GetTypeLabel(chars));

private static string GetTypeLabel<T>(IEnumerable<T> source) => source switch {
    Array array => $"{array.ToString()} is an Array",
    ICollection<T> collection => $"{collection.ToString()} is a Collection",
    _ => "Something else"
};

Array
Collection


If you only want to check a variables type and dont need the declaration part you can use a discard in place of the new declaration name. Since C# 9 you can also completely omit the declaration name. You can see the differnece with `Car` and `Truck` in this example.

In [ ]:
private abstract class Vehicle {}
private class Car : Vehicle {}
private class Truck : Vehicle {}
private double CalculateToll(Vehicle v) => v switch
{
    Car _ => 0.5d,
    Truck => 0.7d,
    null => 1000d,
    _ => 9999d
};
Console.WriteLine(CalculateToll(new Car()));

0,5


## Constant pattern

You can also test a variable to find a match on specific values. The following code shows one example where you test a value against all possible values declared in an enumeration, although of course any constant expression is fine to use, suchs as `char` or `string` literals, `integers` or `floating-point` numericals, `bool`, `const` field or local, or null values. The next example is constant pattern using `not` which is a logical pattern matching negated patterns. This is great for types that overload the `==` operator.

In [ ]:
string? s = "Maybe I'm null";
if (s is not null)
    Console.WriteLine($"s is {s}");

s is Maybe I'm null


And here is another constant pattern example using enums.

In [ ]:
private enum Status
{
    Starting,
    Running,
    Stopping,
    Stopped
}
var e = Status.Running;
private string CheckStaus(Status s) => s switch
{
    Status.Starting => "System is starting up",
    Status.Running => "System is running",
    Status.Stopping => "System is shutting down",
    Status.Stopped => "System is stopped",
    _ => "Invalid status"
};

Console.WriteLine(CheckStaus(e));

System is running


## Relational patterns
You can compare values to constants using relational patterns. This code also shows the conjunctive `and` logical pattern. This combines two or more relational patterns to match. You can also use a disjunctive `or` pattern to check if either match. The parenthesis are only for clarity.

In [ ]:
string State(int temperature) => temperature switch
{
    (> 0) and (< 100) => "liquid",
    < 0 => "solid",
    > 100 => "gas",
    0 => "solid/liquid transition",
    100 => "liquid/gas transition" 
};
Console.WriteLine(State(10));
Console.WriteLine(State(-1));
Console.WriteLine(State(0));

liquid
solid
solid/liquid transition


### Multiple inputs
You don't have to limit yourself to checking one value or property, you can check many too!

In [ ]:
private record Temperature(int Degrees, string Type);
private string State(Temperature t) =>
t switch
{
    (Degrees: (>= 0) and (< 100), Type: "celsius") or (Degrees: (>= 32) and (< 212), Type: "faherenheit") => "liquid",
    (Degrees: < 0, Type: "celsius") or (Degrees: < 32, Type: "faherenheit") => "solid",
    (Degrees: > 100, Type: "celsius") or (Degrees: > 212, Type: "faherenheit") => "gas",
    _ => "I only know celsius and faherenheit"
};
Console.WriteLine(State(new Temperature(2, "celsius")));
Console.WriteLine(State(new Temperature(213, "faherenheit")));
Console.WriteLine(State(new Temperature(19, "celsius")));
Console.WriteLine(State(new Temperature(19, "faherenheit")));


liquid
gas
liquid
solid


## Positional pattern

A record type has a deconstruct method and doesn't have to be used with property names, this is called positional pattern.

In [ ]:
private record Temperature(int Degrees, string Type);
private string State(Temperature t) =>
t switch
{
    ((>= 0) and (< 100), "celsius") or ((>= 32) and (< 212), "faherenheit") => "liquid",
    (< 0, "celsius") or (< 32, "faherenheit") => "solid",
    (> 100, "celsius") or (> 212, "faherenheit") => "gas",
    _ => "I only know celsius and faherenheit"
};
Console.WriteLine(State(new Temperature(2, "celsius")));
Console.WriteLine(State(new Temperature(213, "faherenheit")));
Console.WriteLine(State(new Temperature(19, "celsius")));
Console.WriteLine(State(new Temperature(19, "faherenheit")));

liquid
gas
liquid
solid


## Property pattern

You can use this pattern to match an expressions properties or fields against a nested pattern like this:

In [ ]:
static bool IsDotnetConf(DateTime date) => date is { Year: 2021, Month: 11, Day: 19 or 20 or 21 };
Console.WriteLine(IsDotnetConf(DateTime.Now));

False


You can also combine property pattern type check and add a variable declaration to do something like this:

In [ ]:
static object TakeFive(object input, int n) => input switch
{
    string { Length: >= 5 } s => s.Substring(0, 5),
    string s => s,

    ICollection<char> { Count: >= 5 } symbols => symbols.Take(5).ToArray(),
    ICollection<char> symbols => symbols.ToArray(),

    null => throw new ArgumentNullException(nameof(input)),
    _ => throw new ArgumentException("Not supported input type."),
};
Console.WriteLine(TakeFive("hejsan", 4));
Console.WriteLine(TakeFive(new[] { '1', '2', '3', '4', '5', '6' }));


hejsa
12345


Property patterns are recursive, meaning you can nest any pattern you need to match parts of data. Like this:

In [ ]:
public record Coordinate(int X, int Y);
public record Line(Coordinate Start, Coordinate End);

static bool IsAnyEndOnXAxis(Line segment) =>
    segment is { Start: { Y: 0 } } or { End: { Y: 0 } };

// In C# 10 this can be refactored to:
static bool IsAnyEndOnXAxis10(Line segment) =>
    segment is { Start.Y: 0  } or { End.Y: 0  };

Console.WriteLine(IsAnyEndOnXAxis(new Line(new Coordinate(10, 15), new Coordinate(2,0))));
Console.WriteLine(IsAnyEndOnXAxis10(new Line(new Coordinate(10, 15), new Coordinate(2,10))));

True
False


## Positional pattern
A positional pattern is like a property pattern but the properties are found thanks to a deconstructor instead of property names.

In [ ]:
public readonly struct Coordinate
{
    public int X { get; }
    public int Y { get; }
    public Coordinate(int x, int y) => (X, Y) = (x, y);
    public void Deconstruct(out int x, out int y) => (x, y) = (X, Y);
}

static string Classify(Coordinate c) => c switch
{
    (0, 0) => "Origin",
    (_, >0) => "Positive Y",
    (>0, _) => "Positive X",
    _ => "Something else"
};
Console.WriteLine(Classify(new Coordinate(2, -5)));
Console.WriteLine(Classify(new Coordinate(-8, 10)));

Positive X
Positive Y


Of course, records come with built in deconstructors:

In [ ]:
private record CoordinateRecord(int x, int y);
static string Classify(CoordinateRecord c) => c switch
{
    (0, 0) => "Origin",
    (_, >0) => "Positive Y",
    (>0, _) => "Positive X",
    _ => "Something else"
};
Console.WriteLine(Classify(new CoordinateRecord(2, -5)));
Console.WriteLine(Classify(new CoordinateRecord(-8, 10)));

Positive X
Positive Y


Of course you can also combine all of these patterns in pretty rad ways.

In [ ]:
static decimal GetGroupTicketPriceDiscount(int groupSize, DateTime visitDate)
    => (groupSize, visitDate.DayOfWeek) switch
    {
        (<= 0, _) => throw new ArgumentException("Group size must be positive."),
        (_, DayOfWeek.Saturday or DayOfWeek.Sunday) => 0.0m,
        (>= 5 and < 10, DayOfWeek.Monday) => 20.0m,
        (>= 10, DayOfWeek.Monday) => 30.0m,
        (>= 5 and < 10, _) => 12.0m,
        (>= 10, _) => 15.0m,
        _ => 0.0m,
    };

## Parenthesised pattern
You can put parentheses around any pattern.

In [ ]:
object input =  0;
if (input is not (float or double)) return;

<null>

## `var` Pattern
With the ´var´ pattern you can match any expression, including ´null´ as well as assign it's value to a new local variable. The varn pattern is useful when you hold results of intermediate calculations, like Data() in the example below.

In [ ]:
static List<int> Data() => Enumerable.Range(0, 5).Select(d => new Random().Next(1,10)).ToList<int>();
static bool IsValid() =>
    Data() is var d
    && d.Max() < 10
    && d.Max() > 0;
Console.WriteLine(IsValid());

True


You can also use a ´var´ pattern to do additional checks in a ´when´ case guard of a ´switch´ expression or statement. Like this:

In [ ]:
public record Point(int X, int Y);
static Point Transform(Point point) => point switch
{
    var (x, y) when x < y => new Point(-x, y),
    var (x, y) when x > y => new Point(x, -y),
    var (x, y) => new Point(x, y),
};
Console.WriteLine(Transform(new Point(1, 2)));
Console.WriteLine(Transform(new Point(5, 2)));

Point { X = -1, Y = 2 }
Point { X = 5, Y = -2 }


Or, from an example above...:

In [ ]:
public record Coordinate(int X, int Y);
public record Line(Coordinate Start, Coordinate End);

static bool IsHorizontal(Line segment) =>
    var (s, e) when s.y == e.y;
    //segment is { Start: { Y: 0 } } or { End: { Y: 0 } };

Console.WriteLine(IsHorizontal(new Line(new Coordinate(10, 15), new Coordinate(2,0))));
Console.WriteLine(IsHorizontal(new Line(new Coordinate(10, 15), new Coordinate(2,10))));

Error: (5,16): error CS1002: ; expected
(5,25): error CS1003: Syntax error, '(' expected
(5,25): error CS1001: Identifier expected
(5,31): error CS1001: Identifier expected
(5,31): error CS1026: ) expected